In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import os
import sklearn
import sklearn.neighbors
from pathlib import Path
from pandas.api.types import CategoricalDtype
from operator import itemgetter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer
import torch

import warnings
warnings.filterwarnings('ignore')

# constants
RAW_DATA_PATH = Path('raw_data/')
DATAFRAME_PATH = Path('dataframes/')
MODEL_PATH = Path('model/ncf_model.pt')
TOTAL_TRACKS = 50
NUM_WITHHELD = 25
N_NEIGHBORS = 25
SELECTED_TRACK_FEATURES = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness',                     'liveness', 'valence',
                           'tempo','key', 'loudness',
                          ]

In [2]:
tracks_features_df = pd.read_hdf(DATAFRAME_PATH / 'tracks_features_df.h5', 'tracks_features_df')
cat_type = CategoricalDtype(categories=tracks_features_df.tid, ordered=True)
tracks_features_df.tid = tracks_features_df.tid.astype(cat_type)

playlist_tracks_df = pd.read_hdf(DATAFRAME_PATH/'playlist_tracks_df.h5', 'playlist_tracks_df')
playlist_tracks_df.tid = playlist_tracks_df.tid.astype(cat_type)

playlists_info_df = pd.read_hdf(DATAFRAME_PATH/'playlists_info_df.h5', 'playlists_info_df')

display(tracks_features_df.sample(2))
display(playlist_tracks_df.sample(2))
display(playlists_info_df.sample(2))

,tid,artist_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature
33316,spotify:track:7HwQizneDeLxJmZ9iTmOQV,Blasterjaxx,Astronaut - Original Mix,0.619,0.821,11,-4.706,1,0.0565,0.000317,0.923,0.1060,0.220,127.962,audio_features,264027,4
71027,spotify:track:1q5vlP1KomNe2vOcc3W74R,Tech N9ne,Love Me Tomorrow,0.577,0.764,4,-4.468,0,0.2270,0.035900,0.000,0.0782,0.149,180.112,audio_features,223960,4


,tid,pid,pos
77407,spotify:track:2scewYoYbilBXBQdvsRD73,3132,7
124236,spotify:track:10aWGOqSDBqvNzJ9NeKDbK,4988,36


,pid,playlist_name
4414,9002,ice cream
3283,6614,Classic Rock


In [60]:
# scale features before feeding to the knn model
scaler = MinMaxScaler()
tracks_features_df[SELECTED_TRACK_FEATURES] = pd.DataFrame(scaler.fit_transform(tracks_features_df[SELECTED_TRACK_FEATURES]), columns=SELECTED_TRACK_FEATURES)
tracks_features_df[SELECTED_TRACK_FEATURES]

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,loudness
0,0.914980,0.813,0.125780,0.031225,0.006977,0.0471,0.810,0.569840,0.363636,0.845049
1,0.783401,0.838,0.118503,0.025000,0.025025,0.2420,0.924,0.649683,0.454545,0.896028
2,0.672065,0.758,0.218295,0.002390,0.000000,0.0598,0.701,0.450831,0.181818,0.853389
3,0.902834,0.714,0.146570,0.201807,0.000234,0.0521,0.817,0.458611,0.363636,0.861824
4,0.863360,0.606,0.074116,0.056325,0.000000,0.3130,0.654,0.430392,0.000000,0.885133
...,...,...,...,...,...,...,...,...,...,...
83750,0.558704,0.432,0.032952,0.949799,0.001431,0.2080,0.631,0.321362,0.909091,0.838531
83751,0.351215,0.442,0.039813,0.617470,0.001922,0.1630,0.217,0.796997,0.909091,0.799773
83752,0.535425,0.888,0.084927,0.008133,0.000000,0.2970,0.585,0.417784,0.363636,0.871681
83753,0.694332,0.796,0.031185,0.047189,0.000000,0.1560,0.733,0.513183,0.727273,0.870179


In [61]:
all_playlist_ids = playlist_tracks_df.pid.unique()
train_pids, test_pids = train_test_split(all_playlist_ids,random_state=0, test_size=0.20)
print(f'total playlists: {len(all_playlist_ids)}')
print(f'train playlists: {len(train_pids)}')
print(f'test playlists: {len(test_pids)}')
# 1. Get tracks that are only from the training playlists
# 2. Get the track features(danceability,loudness) for each of these training tracks
# 3. Make a test set that only includes tracks from the training set
train_playlist_tracks_df = playlist_tracks_df.query('pid in @train_pids')

train_tracks_features_df = tracks_features_df.query('tid in @train_playlist_tracks_df.tid')
test_playlist_tracks_df = playlist_tracks_df.query('pid in @test_pids and tid in @train_playlist_tracks_df.tid')
print()
print(f'total tracks in train playlists: {len(train_playlist_tracks_df)}')
print(f'unique tracks in train playlists: {len(train_tracks_features_df)}')


# 1. Get the first NUM_WITHHELD tracks for each playlist in test
# 2. Get the track features for these with held tracks
# 3. Compute mean features by grouping the tracks from incomplete playlists
test_playlist_tracks_incomplete_df = test_playlist_tracks_df.groupby('pid').head(NUM_WITHHELD)
test_tracks_incomplete_features_df = test_playlist_tracks_incomplete_df.merge(tracks_features_df,how='inner',on='tid')
test_playlist_incomplete_features = test_tracks_incomplete_features_df[['pid',*SELECTED_TRACK_FEATURES]].groupby('pid',as_index=False).mean()
print()
print(f'total tracks in test playlists: {len(test_playlist_tracks_df)}')
print(f'total tracks in incomplete test playlists: {len(test_playlist_tracks_incomplete_df)}')
print(f'total incomplete test playlists: {len(test_playlist_incomplete_features)}')

total playlists: 4907
train playlists: 3925
test playlists: 982

total tracks in train playlists: 196250
unique tracks in train playlists: 71774

total tracks in test playlists: 36445
total tracks in incomplete test playlists: 22804
total incomplete test playlists: 971


In [62]:
# KNN model that will find 25 nearest neighbors to the current playlist
knn_clf = sklearn.neighbors.NearestNeighbors(n_neighbors=NUM_WITHHELD)
knn_clf.fit(train_tracks_features_df[SELECTED_TRACK_FEATURES])
distances, indices = knn_clf.kneighbors(test_playlist_incomplete_features[SELECTED_TRACK_FEATURES])

In [63]:
# for each test playlist, get the 25 next nearest predicted tracks and add them to a table for evaluation
def get_predicted_playlist_tracks():
    for index, row in test_playlist_incomplete_features.iterrows():
            predicted_tracks = train_tracks_features_df['tid'].iloc[indices[index]]
            for pos, predicted_track in enumerate(predicted_tracks):
                yield predicted_track, int(row['pid']),pos

test_predicted_playlist_tracks_df = pd.DataFrame(get_predicted_playlist_tracks(), columns =['tid', 'pid', 'pos'])
test_predicted_playlist_tracks_df.tid = test_predicted_playlist_tracks_df.tid.astype(cat_type)

In [72]:
# coode to get 
one_pid = test_playlist_incomplete_features.sample(1)
one_pid

,pid,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,loudness
8,89,0.617328,0.55932,0.040308,0.222839,0.000172,0.145624,0.55904,0.548984,0.501818,0.793481


In [73]:
tracks_in_one_playlist_df = test_predicted_playlist_tracks_df.query('pid in @one_pid.pid')
tracks_info_in_one_playlist_df = tracks_features_df.query('tid in @tracks_in_one_playlist_df.tid')
print('recommendations for ',playlists_info_df.query('pid in @one_pid.pid')['playlist_name'].head(1).values)
tracks_info_in_one_playlist_df = tracks_info_in_one_playlist_df[['artist_name','track_name']]
tracks_info_in_one_playlist_df['distances'] = distances[one_pid.index].reshape(-1,1)
tracks_info_in_one_playlist_df

recommendations for  ["80's"]


,artist_name,track_name,distances
555,Beirut,A Candle’s Fire,0.113290
4386,Renée Elise Goldsberry,Satisfied,0.125549
4650,The Cars,Drive,0.126324
5603,Justin Bieber,Home This Christmas,0.136364
8489,Hadag Nahash,Ezrach Shel Haolam,0.139953
9911,Justin Moore,Small Town USA,0.147621
9996,Phillip Phillips,"Gone, Gone, Gone",0.148770
10392,Tank,"#BDAY (feat. Chris Brown, Siya and Sage The Ge...",0.151290
16045,Bruce Springsteen,Tougher Than the Rest,0.151730
18337,Justin Moore,Bed Of My Chevy,0.153091


In [74]:
class Evaluator:
    def __init__(self,predicted_playlist_tracks,true_playlist_tracks):
        self.predicted_playlist_tracks = predicted_playlist_tracks
        self.true_playlist_tracks = true_playlist_tracks    

    def evaluate(self):
        predicted_playlist_tracks = self.predicted_playlist_tracks
        true_playlist_tracks = self.true_playlist_tracks
        
        r_precision_list = []
        ndcg_list = []
        song_clicks_list = []
        
        pid_list = true_playlist_tracks.pid.unique()
        
        def get_metrics():
            for pid in tqdm(pid_list):
                predictions = predicted_playlist_tracks.query('pid in @pid_list').tid
                truth = true_playlist_tracks.query('pid in @pid_list').tid
                yield (pid , self.r_precision(predictions,truth),self.ndcg(predictions,truth),self.song_clicks(predictions,truth))
        metrics = pd.DataFrame(get_metrics(),columns=['pid','r_precision','ndcg','songs_click'])
        
        return metrics[['r_precision','ndcg','songs_click']].mean()
         
    def r_precision(self,predictions,truth,n_tracks = N_NEIGHBORS):
        truth_set = set(truth)
        prediction_set = set(predictions[:n_tracks])
        intersect = prediction_set.intersection(truth_set)
        return float(len(intersect)) / len(truth_set)
            
    def ndcg(self,predictions,truth,n_tracks = N_NEIGHBORS):
        predictions = list(predictions[:n_tracks])
        truth = list(truth)   
        score = [float(element in truth) for element in predictions]    
        dcg  = np.sum(score / np.log2(1 + np.arange(1, len(score) + 1)))     
        ones = np.ones([1,len(truth)])
        idcg = np.sum(ones / np.log2(1 + np.arange(1, len(truth) + 1)))
        return (dcg / idcg)
    
    def song_clicks(self,predictions,truth,n_tracks = N_NEIGHBORS):
        predictions = predictions[:n_tracks]
        i = set(predictions).intersection(set(truth))
        for index, t in enumerate(predictions):
            for track in i:
                if t == track:
                    return float(int(index / 10))              
        return float(n_tracks / 10.0 + 1)

In [75]:
model_eval = Evaluator(test_predicted_playlist_tracks_df,test_playlist_tracks_df)
model_eval.evaluate()

100%|████████████████████████████████████████████████████████████████████████████████| 971/971 [00:27<00:00, 35.07it/s]


r_precision    0.000391
ndcg           0.000676
songs_click    0.000000
dtype: float64

## Neural Network (NeuMF)

In [13]:
from scipy.sparse import dok_matrix

In [14]:
total_playlist_tracks_df = train_playlist_tracks_df.append(test_playlist_tracks_incomplete_df)
unique_tracks = total_playlist_tracks_df['tid'].unique()
total_cat_type = CategoricalDtype(categories=unique_tracks, ordered=True)
total_playlist_tracks_df['tid'] = total_playlist_tracks_df.tid.astype(total_cat_type)
total_playlist_tracks_df['cat_codes'] = total_playlist_tracks_df['tid'].cat.codes

dok_mat_n_rows = total_playlist_tracks_df.shape[0]
dok_mat_n_cols = len(unique_tracks)

dok_mat_rows = total_playlist_tracks_df['pid']
dok_mat_cols = total_playlist_tracks_df['cat_codes']

# Make a dictionary key sparse matrix
dok_mat = dok_matrix((dok_mat_n_rows, dok_mat_n_cols))

# TODO vectorize later if possible
for (pid, cat_code) in tqdm(zip(dok_mat_rows, dok_mat_cols)):
    dok_mat[pid, cat_code] = 1.0

219054it [00:03, 55781.33it/s]


In [15]:
rand_negative_fill_in = 4
layer_sizes = [64, 32, 16, 8]

# Hyperparameters
embedding_dim = 8
num_epochs = 2
learning_rate = 0.001
batch_size = 200

In [16]:
class NeuralMF(torch.nn.Module):
    def __init__(self, num_pl, num_tr, dim=embedding_dim):
        super(NeuralMF, self).__init__()
        num_of_layers = len(layer_sizes)
            
        self.pl_embedding = torch.nn.Embedding(num_pl, dim)
        self.tr_embedding = torch.nn.Embedding(num_tr, dim)
        self.pl_mlp_embedding = torch.nn.Embedding(num_pl, int(layer_sizes[0]/2))
        self.tr_mlp_embedding = torch.nn.Embedding(num_tr, int(layer_sizes[0]/2))
            
        self.mlp = torch.nn.ModuleList()
        for i in range(1, num_of_layers):
            self.mlp.append(torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i]))
            self.mlp.append(torch.nn.ReLU())

        self.affine_final = torch.nn.Linear(dim + layer_sizes[-1], 1)
        self.logistic_sig = torch.nn.Sigmoid()
    
    def init_weight(self):
        torch.nn.init.normal_(self.pl_embedding, std=0.01)
        torch.nn.init.normal_(self.tr_embedding, std=0.01)
        torch.nn.init.normal_(self.pl_mlp_embedding, std=0.01)
        torch.nn.init.normal_(self.tr_mlp_embedding, std=0.01)
        
        for layer in self.mlp:
            if isinstance(layer, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(layer.weight)
                
        torch.nn.init.xavier_uniform_(self.affine_final.weight)
        
        for mod in self.modules():
            if isinstance(mod, torch.nn.Linear) and mod.bias is not None:
                mod.bias.data.zero_()
        
    def forward(self, playlists, tracks):
        pl_vec = self.pl_embedding(playlists)
        tr_vec = self.tr_embedding(tracks)
        prod = torch.mul(pl_vec, tr_vec)
        
        pl_mlp_vec = self.pl_mlp_embedding(playlists)
        tr_mlp_vec = self.tr_mlp_embedding(tracks)
        mlp_vec = torch.cat([pl_mlp_vec, tr_mlp_vec], dim=-1)
        
        for i, layer in enumerate(self.mlp):
            mlp_vec = layer(mlp_vec)
            
        result = self.affine_final(torch.cat([prod,mlp_vec], dim=-1))
        activated_result = self.logistic_sig(result)
        return torch.flatten(activated_result)

In [17]:
model = NeuralMF(dok_mat.shape[0], dok_mat.shape[1])
loss_fn = torch.nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

def plot_loss(iters, losses):
    plt.plot(iters, losses)
    plt.title(f'Training Curve (batch_size={batch_size}, lr={learning_rate})')
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

def make_train_data():
    for (pl, tr) in dok_mat.keys():
        yield pl, tr, 1.0
        for t in range(rand_negative_fill_in):
            rand_num = np.random.randint(dok_mat.shape[1])
            while(pl, rand_num) in dok_mat.keys():
                rand_num = np.random.randint(dok_mat.shape[1])
            yield pl, rand_num, 0.0

def train_loop(data_loader):
    losses = []; iters = []; offset = 0
    for epoch in range(num_epochs):
        for idx, (pl, tr, recs) in enumerate(tqdm(data_loader)):
            recs = recs.float()
            optim.zero_grad()
            out = model(pl,tr)
            loss = loss_fn(out, recs)
            losses.append(loss)
            iters.append(offset+idx)
            loss.backward()
            optim.step()
        offset = iters[-1]
        print(f'training loss after epoch-{epoch+1} = {(losses[-1]):.4f}')
    return iters, losses

def run_training():
    data_loader = torch.utils.data.DataLoader(list(make_train_data()), batch_size=batch_size)
    return train_loop(data_loader)


In [18]:
iters, losses = run_training()
torch.save(model.state_dict(), MODEL_PATH)
plot_loss(iters, losses)

100%|██████████████████████████████████████████████████████████████████████████████| 5433/5433 [07:42<00:00, 11.74it/s]


training loss after epoch-1 = 0.4290


In [19]:
model.load_state_dict(torch.load(MODEL_PATH)) #👌 

<All keys matched successfully>

## Make recommendations using the NeuMF model

In [20]:
playlist_for_test = np.random.choice(test_playlist_tracks_df['pid'].tolist())
ground_truth = test_playlist_tracks_df[test_playlist_tracks_df['pid'] == playlist_for_test]

playlist_embedding_weight_matrix = model.pl_mlp_embedding.weight
chosen_playlist_vector = playlist_embedding_weight_matrix[playlist_for_test]

In [ ]:
# Hyperparameter Search
# NOTE: WILL TAKE AGES TO RUN
km_model = KMeans()
visualizer = KElbowVisualizer(km_model, k=(50,100))
visualizer.fit(playlist_embedding_weight_matrix.detach().numpy())
visualizer.show()

In [22]:
print("Fitting a KMeans model with 100 clusters to the embedding weight matrix for playlists")
km_model = KMeans(n_clusters=100, random_state=0, verbose=0).fit(playlist_embedding_weight_matrix.detach().numpy())

Fitting a KMeans model with 100 clusters to the embedding weight matrix for playlists


In [23]:
chosen_playlist_vector = chosen_playlist_vector.detach().numpy().reshape(1,-1)
playlist_predictor = km_model.predict(chosen_playlist_vector)
playlist_labels = km_model.labels_

similar_playlists = []
for pid, playlist_label in enumerate(playlist_labels):
    if playlist_label == playlist_predictor:
        similar_playlists.append(pid)
print(f'other playlists in cluster: {len(similar_playlists)}')

other playlists in cluster: 2223


In [24]:
from collections import OrderedDict
tracks = []
for pid in similar_playlists:
    tracks += list(total_playlist_tracks_df[total_playlist_tracks_df['pid'] == pid]['cat_codes'])
print(f'other tracks from similar_playlists in cluster: {len(tracks)}') 

tracks = list(OrderedDict.fromkeys(tracks))

pids_pred = torch.tensor(np.full(len(tracks), playlist_for_test, dtype='int32'))
tracks_pred = torch.tensor(np.array(tracks, dtype='int32'))

results = model(pids_pred, tracks_pred)
print("Retrieved predictions from trained model...")

other tracks from similar_playlists in cluster: 2300
Retrieved predictions from trained model...


In [28]:
unique_playlist_tracks_df = total_playlist_tracks_df.drop_duplicates(subset=['cat_codes'])
unique_playlist_tracks_df = unique_playlist_tracks_df.sort_values(by=['cat_codes'], ascending=False)

unique_tids_by_cat_codes = np.array(unique_playlist_tracks_df['tid'])
unique_result_merged = unique_playlist_tracks_df.merge(tracks_features_df, on='tid')
other_features_by_cat_codes = np.array([(row['track_name'],row['artist_name']) for i, row in unique_result_merged.iterrows()])

def get_results_df():
    for i, probability in enumerate(tqdm(results)):
        tid = unique_tids_by_cat_codes[i]
        other_features = other_features_by_cat_codes[i]
        yield probability.item(), tid, *other_features

results_df = pd.DataFrame(get_results_df(), index=range(len(results)), columns=['probability', 'tid','track_name', 'artist_name'])
results_df = results_df.sort_values(by=['probability'], ascending=False)

nmf_preds = results_df.head(NUM_WITHHELD)
nmf_preds

100%|██████████████████████████████████████████████████████████████████████████| 2110/2110 [00:00<00:00, 140656.74it/s]


,probability,tid,track_name,artist_name
1495,0.861108,spotify:track:2L9FQLoRs0Fz0Ih2j4qkJb,#1 Fan (feat. Keyshia Cole & J. Holiday),Plies
1674,0.859154,spotify:track:5H4fWHh89y9qm54f5309RZ,"Piano Sonata No.9 in D, K.311: 2. Andantino co...",Wolfgang Amadeus Mozart
1796,0.856589,spotify:track:6HCNagyWwHLjNBQ2c6UBox,And On,The Land Below
22,0.856555,spotify:track:5R3YBO6IgrilnZlxnfJFZS,Blues Blues,Bo Diddley
274,0.855106,spotify:track:0m0DAL7AJc6oo74D1OtaVJ,The Hands That Thieve,Streetlight Manifesto
321,0.854513,spotify:track:3tKPgVWDxd4UqqhNklqGqC,Beneath the Canopy,Nicole Vaughn
1714,0.852546,spotify:track:72mx4tVoM7knI9D20trABp,Ruby Baby,Donald Fagen
547,0.850232,spotify:track:4s54h9NeCfr73BxtQhQw7h,Trap Queen,Piano Dreamers
1827,0.849188,spotify:track:4cPYf8lBQIfu2pF0AozESv,Her Snowfall Was A Line Of Cocaine,Lady Radiator
1830,0.847725,spotify:track:4SON4XSisFbnMufSoSwoyD,Pack Of Thieves,Enter Shikari


In [29]:
def predictions_nmf():
    for idx,row in nmf_preds.iterrows():
        yield (row['tid'], playlist_for_test, idx)

predictions_nmf_df = pd.DataFrame(predictions_nmf(), columns=['tid','pid', 'pos'])
nmf_model_eval = Evaluator(predictions_nmf_df, ground_truth.iloc[25:])
nmf_model_eval.evaluate()

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.09it/s]


r_precision    0.0
ndcg           0.0
songs_click    3.5
dtype: float64

In [32]:
test_playlist_tracks_incomplete_df.query('pid == @playlist_for_test')[['tid','pos']].merge(tracks_features_df[['tid','artist_name','track_name']], on='tid')

,tid,pos,artist_name,track_name
0,spotify:track:56twMOlq9DJc3EgpVUVIrW,0,Demi Lovato,Stone Cold
1,spotify:track:7hsLKGEnoiNShdIGL6ws1f,1,AWOLNATION,Sail
2,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,2,Donnie Trumpet & The Social Experiment,Sunday Candy
3,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,3,DRAM,Broccoli (feat. Lil Yachty)
4,spotify:track:5xV0Czdqefft6sPDqjmFBu,4,Mario,Let Me Love You
5,spotify:track:5Q0Nhxo0l2bP3pNjpGJwV1,5,Miley Cyrus,Party In The U.S.A.
6,spotify:track:5oQcOu1omDykbIPSdSQQNJ,6,Bowling For Soup,1985
7,spotify:track:0p1BcEcYVO3uk4KDf3gzkY,7,Hunter Hayes,Wanted
8,spotify:track:4oa14QBfWRDfJy2agySy0L,8,Sara Bareilles,Gravity
9,spotify:track:75JFxkI2RXiU7L9VXzMkle,9,Coldplay,The Scientist


In [33]:
ground_truth[['tid','pos']].merge(tracks_features_df[['tid','artist_name','track_name']], on='tid')

,tid,pos,artist_name,track_name
0,spotify:track:56twMOlq9DJc3EgpVUVIrW,0,Demi Lovato,Stone Cold
1,spotify:track:7hsLKGEnoiNShdIGL6ws1f,1,AWOLNATION,Sail
2,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,2,Donnie Trumpet & The Social Experiment,Sunday Candy
3,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,3,DRAM,Broccoli (feat. Lil Yachty)
4,spotify:track:5xV0Czdqefft6sPDqjmFBu,4,Mario,Let Me Love You
5,spotify:track:5Q0Nhxo0l2bP3pNjpGJwV1,5,Miley Cyrus,Party In The U.S.A.
6,spotify:track:5oQcOu1omDykbIPSdSQQNJ,6,Bowling For Soup,1985
7,spotify:track:0p1BcEcYVO3uk4KDf3gzkY,7,Hunter Hayes,Wanted
8,spotify:track:4oa14QBfWRDfJy2agySy0L,8,Sara Bareilles,Gravity
9,spotify:track:75JFxkI2RXiU7L9VXzMkle,9,Coldplay,The Scientist
